# **Import Statements:**

In [2]:
from datasets import load_dataset
from google.colab import drive
import torch

drive.mount('/content/drive')
%cd "/content/drive/My Drive/CS196Project/Group6"
torch.cuda.empty_cache()

ModuleNotFoundError: ignored

In [3]:
%%bash
# pip install pytorch_lightning
# pip install gitpython
# pip install rouge-score
# pip install sacrebleu
pip install datasets
# pip install transformers
# pip install lightning_base

cd "/content/drive/My Drive/CS196Project/Group6/finetuning"
pip install -r requirements.txt

pip install jupyter_http_over_ws
jupyter serverextension enable --py jupyter_http_over_ws

  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


bash: line 9: cd: /content/drive/My Drive/CS196Project/Group6/finetuning: No such file or directory
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-serverextension", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/serverextensions.py", line 293, in start
    super(ServerExtensionApp, self).start()
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 256, in start
    self.subapp.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/serverextensions.py", line 210, in start
    self.toggle_s

# **Load Datasets:**

 TODO: format newsroom
 
 TODO: format custom dataset

In [4]:
newsroom = load_dataset('newsroom', data_dir='/content/drive/My Drive/CS196Project/newsroom')

Using custom data configuration default


Dataset newsroom downloaded and prepared to /root/.cache/huggingface/datasets/newsroom/default/1.0.0/4b405ccd64e15f685065870ea563a1e6a034d1bd269a5427f40146d81549095e. Subsequent calls will reuse this data.


# **Testing Transformers**

In [ ]:
%%bash
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/finetune.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/callbacks.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/utils.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/sentence_splitter.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/lightning_base.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/requirements.txt


mkdir example_data && cd example_data

wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/test.source
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/test.target
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/train.source
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/train.target
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/val.source
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/val.target

cd ..

In [1]:
%%bash

cd "/content/drive/My Drive/CS196Project/Group6/finetuning/"

    --do_predict \
    --eval_batch_size=1 \

export OUTPUT_DIR="example_output"

python finetune.py \
    --learning_rate=3e-5 \
    --fp16 \
    --gpus 1 \
    --do_train \
    --n_val 100 \
    --val_check_interval 0.1 \
    --data_dir="/content/drive/My Drive/CS196Project/Group6/finetuning/example_data" \
    --train_batch_size=1 \
    --output_dir=$OUTPUT_DIR \
    --freeze_embeds \
    --freeze_encoder \
    --num_train_epochs 6 \
    --max_target_length=20 \
    --val_max_target_length=60 \
    --test_max_target_length=20 \
    --model_name_or_path facebook/bart-large
    "$@"

bash: line 2: cd: /content/drive/My Drive/CS196Project/Group6/finetuning/: No such file or directory
bash: line 5: --do_predict: command not found
python3: can't open file 'finetune.py': [Errno 2] No such file or directory


--2020-11-04 05:46:39--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/val.source
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9589 (9.4K) [text/plain]
Saving to: ‘val.source’

val.source          100%[===================>]   9.36K  --.-KB/s    in 0.001s  

2020-11-04 05:46:39 (7.17 MB/s) - ‘val.source’ saved [9589/9589]

--2020-11-04 05:46:39--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/val.target
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting re